In [ ]:
%pylab inline

In [ ]:
import os
import numpy as np
import trimesh
import pyrender
import matplotlib.pyplot as plt
import json
import cv2
import re
import glob
from tqdm import tqdm 
from skimage.io import imread, imsave
from PIL import Image
import shutil 



In [ ]:

def run_meshroom(frames_path, output_path, meshroom_base, run_meshing=False):
    
    if not os.path.exists(meshroom_base):
        print("Could not find meshroom base path ( where you downloaded to ) ")
        return

    #path = os.path.abspath(__file__)
    #base = os.path.dirname(path)

    graph_name = "meshroom_graph_all.mg"
    
    if run_meshing == False:
        graph_name = "meshroom_graph_no_mesh.mg"

    graph_path = graph_name #os.path.join(base, graph_name)

    assert os.path.exists(graph_path), "Could not find meshroom graph file"

    frames_path = os.path.abspath(frames_path)
    output_path = os.path.abspath(output_path)
    
    exe_path = os.path.join(meshroom_base, "meshroom_photogrammetry.exe")

    kwargs = {
        "exe_path" : exe_path,
        "input" : frames_path,
        "cache" : output_path, 
        "output" : output_path,
        "pipeline" : graph_path,
    }

    cmd = "{exe_path} --input {input} --cache {cache} --pipeline {pipeline} --output {output}".format(**kwargs)
    print("[Meshroom] Running cmd: ", cmd)

    os.system(cmd)

In [ ]:

def copy_meshroom_files_and_cleanup(meshroom_base, to_path):

    sfm_path = os.path.join(meshroom_base, "StructureFromMotion")

    if not os.path.exists(sfm_path):
        print("Couldnt find MeshroomCache folder - meshroom/StructureFromMotion ")
        return

    def SubDirPath (d):
        return filter(os.path.isdir, [os.path.join(d,f) for f in os.listdir(d)])

    def LatestDirectory (d):
        return max(SubDirPath(d), key=os.path.getmtime)

    
    cameras_path = os.path.join(LatestDirectory(sfm_path), "cameras.sfm")
    cloud_path = os.path.join(LatestDirectory(sfm_path), "cloud_and_poses.abc")

    export_path = os.path.join(meshroom_base, "ExportAnimatedCamera")
    undist_images_path = os.path.join(LatestDirectory(export_path), "undistort")


    if not os.path.exists(cameras_path):
        print("Couldnt find cameras.sfm - /StructureFromMotion/UUID/cameras.sfm ")
        return

    if not os.path.exists(undist_images_path):
        print("Couldnt find undistorted images folder - ExportAnimatedCamera/UUID/undistort")
        return


    try:
        #ConvertSfMFormat / sfm.json
        convert_path = os.path.join(meshroom_base, "ConvertSfMFormat")
        sfm_json_path = os.path.join(LatestDirectory(convert_path), "sfm.json")
        shutil.copy2( sfm_json_path, to_path )
    except Exception as e:
        print("Error copying sfm data: ", str(e))
    
    # Texturing optional
    texturing_base = os.path.join(meshroom_base, "Texturing")
    if os.path.exists(texturing_base):
        texturing_base = LatestDirectory(texturing_base)
        # Copy textured .obj file 
        files = ["texturedMesh.obj", "texturedMesh.mtl"]
        for file in files:
            tex_path = os.path.join(texturing_base, file)
            if os.path.exists( tex_path ):
                shutil.copy2( tex_path, to_path)
            else:
                print(" [Warning] Could not find mesh file: ", file )

        textures = glob.glob( os.path.join(texturing_base, "*.png") )
        textures += glob.glob( os.path.join(texturing_base, "*.jpg") )
        
        for tex_file in textures:
            shutil.copy2( tex_file, to_path )


    shutil.copy2( cameras_path, to_path )

    if os.path.exists(cloud_path):
        shutil.copy2( cloud_path, to_path )

    try:
        shutil.copytree(undist_images_path, os.path.join(to_path, "undistorted_frames"))
    except Exception as e:
        print("Error copying undistorted frames: \n", str(e))
        pass

In [ ]:

def get_video_frames_gen(video_path):
    capture = cv2.VideoCapture(video_path)
    while True:
        read_flag, frame = capture.read()
        if not read_flag:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        yield frame

    capture.release()

    
def extract_frames(video_path, out_path, step=1):
    
    video_generator = get_video_frames_gen(video_path)
    frame_out_idx = 0
    
    for frame_idx, image_arr in enumerate(video_generator):
        if frame_idx % step != 0:
            continue
        frame_name = "frame_{:05d}.jpg".format(frame_out_idx)
        frame_out_path = os.path.join(out_path, frame_name)
        imsave(frame_out_path, image_arr)
        frame_out_idx += 1


In [ ]:
def alice_to_gl(pose_in):
    pose_out = pose_in.copy()
    
    # X 
    pose_out[1,0] = pose_in[0,1]
    pose_out[2,0] = pose_in[0,2]
    
    # Y 
    pose_out[0,1] = pose_in[1,0] * -1.0
    pose_out[1,1] = pose_in[1,1] * -1.0 # flip Y 
    pose_out[2,1] = pose_in[1,2] * -1.0 
    
    # Z 
    pose_out[0,2] = pose_in[2,0] * -1.0
    pose_out[1,2] = pose_in[2,1] * -1.0 
    pose_out[2,2] = pose_in[2,2] * -1.0
    
    return pose_out
    
    
def get_pose_mat(p_dict):
    trans = p_dict['pose']['transform']
    translation = trans['center']
    rot = trans['rotation']
    mat = np.eye(4, dtype=np.float64)
    #mat[:3,:3] = np.array(rot).reshape((3,3))
    mat[:3, 0] = rot[0:3]
    mat[:3, 1] = rot[3:6]
    mat[:3, 2] = rot[6:9]
    mat[:3, 3] = translation 
    return mat

def convert_rt_to_opengl(rvec, tvec):
    # https://answers.opencv.org/question/23089/opencv-opengl-proper-camera-pose-using-solvepnp/
    
    view_matrix = np.zeros((4,4), np.float32)
    rvec = rvec.astype(np.float32)
    assert rvec.shape[0] == 3

    rotation, jacobian = cv2.Rodrigues(rvec)
    
    view_matrix[:3,:3] = rotation
    view_matrix[:3, 3] = tvec[:,0]
    view_matrix[3,3] = 1.0
    
    view_matrix = np.linalg.inv(view_matrix)
    view_matrix[:,1] *= -1.0
    view_matrix[:,2] *= -1.0
    
    return view_matrix 


def get_projection_matrix(width, height, yfov_radians, znear=0.01, zfar=1000.0):
    """Return the OpenGL projection matrix for this camera.

    Parameters
    ----------
    width : int
        Width of the current viewport, in pixels.
    height : int
        Height of the current viewport, in pixels.
    """

    aspect_ratio = float(width) / float(height)

    a = aspect_ratio
    t = np.tan(yfov_radians / 2.0)
    n = znear
    f = zfar

    P = np.zeros((4,4))
    P[0][0] = 1.0 / (a * t)
    P[1][1] = 1.0 / t
    P[3][2] = -1.0

    if f is None:
        P[2][2] = -1.0
        P[2][3] = -2.0 * n
    else:
        P[2][2] = (f + n) / (n - f)
        P[2][3] = (2 * f * n) / (n - f)

    return P

## Run Meshroom 

In [ ]:
meshroom_exe_base_path = "D:\\Meshroom-2019.1.0" 

In [ ]:
video_path = "IMG_6097.MOV"
frame_step = 8
run_meshing = True 

In [ ]:
video_name, ext = os.path.splitext(os.path.basename(video_path))
video_base = os.path.dirname(video_path)
video_data_path = os.path.join(video_base, video_name + "_out")

### This takes a while 
#### NOTE: even if 'run_meshing' is True, this step can still fail to generate a mesh
This happens when certain images aren't processed 

#### NOTE:  delete any existing output if running this multiple times 

In [ ]:
frames_out_path = os.path.join(video_data_path, "frames")

os.makedirs(frames_out_path)

print(" [Extracting Frames] ", video_path , frames_out_path)

extract_frames(video_path, frames_out_path, step=frame_step)

meshroom_path = os.path.join(video_data_path, "meshroom")


os.makedirs(meshroom_path)


run_meshroom(frames_out_path, meshroom_path, 
             meshroom_base=meshroom_exe_base_path, 
             run_meshing=run_meshing)


copy_meshroom_files_and_cleanup(meshroom_path, video_data_path)


In [ ]:
camera_poses = json.load(open( os.path.join( video_data_path , "cameras.sfm") , 'r'))

In [ ]:

undist_base = os.path.join(video_data_path, "undistorted_frames")
undist_images = sorted(glob.glob(os.path.join(undist_base, "*.jpg")))


In [ ]:
image = imread(undist_images[0])

In [ ]:
obj_file = os.path.join(video_data_path, "texturedMesh.obj")


In [ ]:
obj_scene = trimesh.load(obj_file)
# this used to return Trimesh ? 

In [ ]:
#obj_scene.show()

In [ ]:
if isinstance(obj_scene, trimesh.Trimesh):
    meshes = [obj_scene]
else:
    meshes = [ v for k,v in obj_scene.geometry.items() ]

In [ ]:
obj_mesh = pyrender.Mesh.from_trimesh(meshes)

In [ ]:
obj_mesh.primitives[0].material.doubleSided = True

In [ ]:
views = camera_poses['views']
pose_dicts = camera_poses['poses']
intrinsics = camera_poses['intrinsics']

In [ ]:
intrinsics

In [ ]:
img_w = int(intrinsics[0]['width'])

In [ ]:
img_w

In [ ]:
px,py = map(float, intrinsics[0]['principalPoint'])

In [ ]:
focal_length = float(intrinsics[0]['pxFocalLength'])

In [ ]:
img_h, img_w = image.shape[:2]
print(img_w, img_h)

In [ ]:
yfov_radians = 2.0 * arctan( (img_h / 2.0) / focal_length )

In [ ]:
def view_for_image(image_name, views):
    view = None 
    for v in views:
        if os.path.basename(v['path']) == image_name:
            return v
    return None


In [ ]:
id_to_poses = {}
id_to_pose_dict = {}
for p in pose_dicts:
    pose_id = p['poseId']
    id_to_poses[pose_id] = get_pose_mat(p)
    id_to_pose_dict[pose_id] = p.copy()
    

## Render 3d Mesh overlaid onto images 

In [ ]:
scene = pyrender.Scene(ambient_light=[1.15, 1.15, 1.15],
                       bg_color=[0.0, 0.0, 0.0])

camera = pyrender.PerspectiveCamera(yfov = yfov_radians)

camera_node = pyrender.Node(camera=camera, matrix=np.eye(4))
scene.add_node(camera_node)

scene.add(obj_mesh)

light = pyrender.PointLight(intensity=1.3)
scene.add(light, pose=np.eye(4))

renderer = pyrender.OffscreenRenderer(img_w, img_h)

In [ ]:

mat = obj_mesh.primitives[0].material
mat.wireframe = True


In [ ]:
mat.baseColorFactor = np.array([1,1,1,1], dtype=np.float)

In [ ]:

video_out_path = os.path.join(video_data_path, "viz.mp4")

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video_out = cv2.VideoWriter(video_out_path, fourcc, 24.0, (img_w, img_h))

num_frames = len(undist_images)

depth_alpha = None


for i, undist_path in tqdm(enumerate(undist_images), total=num_frames):
    
    undist_image_name = os.path.basename(undist_path)    
    input_image = imread(undist_path)
    
    #print(undist_image_name)
    
    # These are named 1342558130_frame_000001.jpg hopefully that's consistent 
    image_name = re.findall(r'(\d+)_(.*)', undist_image_name)[0][1]
    
    view = view_for_image(image_name, views)
    pose_id = view['poseId']
    
    if pose_id not in id_to_poses:
        print("Missing pose for image: ", image_name)
        continue
        
    pose_alice = id_to_poses[pose_id]
    pose_gl = alice_to_gl(pose_alice)
    
    projection = get_projection_matrix(img_w, img_h, yfov_radians)
    view_matrix = np.linalg.inv(pose_gl)
    mvp = np.dot(projection, view_matrix)
    
    ###
    
    scene.set_pose(camera_node, pose_gl)
    color_img, depth = renderer.render(scene)
    
    #print( " Depth max: {:5.3f}  min {:5.3f}".format( depth.min(), depth.max() ))
    
    max_depth = 0.75
    
    depth_norm = (255.0 * (depth / max_depth)).astype(np.uint8)
    depth_color = cv2.applyColorMap(depth_norm, cv2.COLORMAP_HSV)
    
    progress = i / float(num_frames-1)
    
    if depth_alpha is None:
        depth_alpha = np.zeros_like(input_image, dtype=np.float)
        h,w = input_image.shape[:2]
    
    depth_alpha.fill(0)
    row_end = int(round(  h * np.sin( progress * np.pi * 0.5)))
    depth_alpha[:row_end, :, :] = 1.0 
        
#     depth_alpha = (float(i) / float( min(140,num_frames) ))
#     depth_alpha = 0.9 * min(1.0, depth_alpha)
#     depth_alpha = (i / float(num_frames)) * 2.0
    
    output = depth_color.astype(np.float) * depth_alpha + input_image.astype(np.float) * (1.0 - depth_alpha)
    
    #color_img = cv2.cvtColor(color_img, cv2.COLOR_RGB2BGR)
    
    video_out.write(  cv2.cvtColor(output.astype(np.uint8), cv2.COLOR_RGB2BGR)  )
    
    
video_out.release() 
video_out = None 

